In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time, sys, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
# For Tensorflow & Keras
import tensorflow as tf
import keras.layers as lyr
from keras.models import Model
#For Padding
from keras.preprocessing.sequence import pad_sequences
#For XGBoost
import xgboost as xgb

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# LOAD DATA
X_train, X_traintest, Y_train, Y_traintest = splitData()
X_test, test_idxs = testData()

# 1st Stage, Question to Int Array
# Build Vocabulary

f1 = BoW()
f1.fit(X_train.append(X_traintest))
X_all = X_train.append(X_traintest)
mycv = f1.getVectorizer()
#words to tokens
X_test_q1 = create_padded_seqs(X_test.question1,mycv)
X_test_q2 = create_padded_seqs(X_test.question2, mycv)
X_train_q1 = create_padded_seqs(X_train.question1, mycv)
X_train_q2 = create_padded_seqs(X_train.question2, mycv)
X_traintest_q1 = create_padded_seqs(X_traintest.question1, mycv)
X_traintest_q2 = create_padded_seqs(X_traintest.question2, mycv)

SPLIT DATA
LOAD TRAIN DATA
LOAD TEST DATA
FIT CORPUS ON COUNT VECTORIZER


In [8]:
len(mycv.vocabulary_)

5000

In [72]:
# Feed NN with Padded Sequences

input1_tensor = lyr.Input(X_train_q1.shape[1:])
input2_tensor = lyr.Input(X_train_q2.shape[1:])

#words_embedding_layer = lyr.Embedding(X1_train_q1.max() + 1, 100)
merge_layer = lyr.multiply([input1_tensor, input2_tensor])
ouput_layer = lyr.Dense(1, activation='sigmoid')(merge_layer)
model = Model([input1_tensor, input2_tensor], ouput_layer)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 10)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 10)            0                                            
____________________________________________________________________________________________________
multiply_2 (Multiply)            (None, 10)            0           input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             11          multiply_2[0][0]        

In [73]:
# FIT --> Train NN weights
model.fit([X_train_q1, X_train_q2], Y_train, 
          validation_data=([X_traintest_q1, X_traintest_q2], Y_traintest), 
          batch_size=128, epochs=6, verbose=2)

# (1) Take Features from Merge Layer and(2) feed to Classifier (XGBoost)
#(1)
mergeNN = Model([input1_tensor, input2_tensor], merge_layer)
mergeNN.compile(loss='mse', optimizer='adam')

F_train = mergeNN.predict([X_train_q1, X_train_q1], batch_size=128)
F_traintest = mergeNN.predict([X_traintest_q1, X_traintest_q2], batch_size=128)
F_test = mergeNN.predict([X_test_q1, X_test_q2], batch_size=128)

#(2)
dTrain = xgb.DMatrix(F_train, label=Y_train)
dTraintest = xgb.DMatrix(F_traintest, label=Y_traintest)
dTest = xgb.DMatrix(F_test)

xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.1, 
    'max_depth': 9,
    'subsample': 0.9,
    'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'min_child_weight': 5,
    'silent': 1
}
bst = xgb.train(xgb_params, dTrain, 1000,  [(dTrain,'train'), (dTraintest,'val')], 
                verbose_eval=10, early_stopping_rounds=10)


Train on 283001 samples, validate on 121287 samples
Epoch 1/6
2s - loss: 9.7523 - val_loss: 9.7517
Epoch 2/6
2s - loss: 9.7523 - val_loss: 9.7517
Epoch 3/6
2s - loss: 9.7523 - val_loss: 9.7517
Epoch 4/6
2s - loss: 9.7523 - val_loss: 9.7517
Epoch 5/6
2s - loss: 9.7523 - val_loss: 9.7517
Epoch 6/6
2s - loss: 9.7523 - val_loss: 9.7517
[0]	train-logloss:0.682886	val-logloss:0.683174
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 10 rounds.
[10]	train-logloss:0.633481	val-logloss:0.63598
[20]	train-logloss:0.611799	val-logloss:0.617023
[30]	train-logloss:0.597875	val-logloss:0.605675
[40]	train-logloss:0.589812	val-logloss:0.599589
[50]	train-logloss:0.580111	val-logloss:0.592039
[60]	train-logloss:0.573041	val-logloss:0.586838
[70]	train-logloss:0.567315	val-logloss:0.582681
[80]	train-logloss:0.561625	val-logloss:0.578658
[90]	train-logloss:0.557021	val-logloss:0.575543
[100]	train-logloss:0.552771	va

In [74]:
# Predict Validation Set and Compute Log Loss 
X_val = xgb.DMatrix(F_traintest)
Y_pred = bst.predict(X_val, ntree_limit=bst.best_ntree_limit)
Y_true = Y_traintest
print log_loss(Y_true, Y_pred)
# PREDICT Submission File
dTest = xgb.DMatrix(F_test)
df_sub = pd.DataFrame({
        'test_id': test_idxs,
        'is_duplicate': bst.predict(dTest, ntree_limit=bst.best_ntree_limit)
    }).set_index('test_id')


print("Create Submission File")
df_sub.to_csv('newsub.csv')

0.507999677833
Create Submission File


In [75]:
len(df_sub)

2345790

# Helper Functions

In [5]:
def create_padded_seqs(texts, mycv):
    seqs = texts.apply(text2ints, cv=mycv)
    return pad_sequences(seqs, maxlen=10)

def text2ints(text, cv):
    other_index = len(cv.vocabulary_)
    intseq = []
    mysplit = text.split(" ")
    filter(lambda x: x!="", mysplit)
    for word in mysplit:
        intseq.append(cv.vocabulary_[word]) if word in cv.vocabulary_ else other_index
    return intseq

#Train_Path = './input/df10.csv' # JUST TO TEST FASTER , normally train.csv
#Test_Path = './input/df10.csv'
Train_Path = './input/train.csv'
Test_Path = './input/test.csv'

def cleanreview(review):
    letters_only = re.sub("[^a-z]", " ", review.lower())
    return letters_only
def cleanquestions(df):

    question1 = df.question1.apply(lambda x: cleanreview(x))
    question2 = df.question2.apply(lambda x: cleanreview(x))
    df.loc[:, 'question1'] = pd.Series(question1, index=df.index)
    df.loc[:, 'question2'] = pd.Series(question1, index=df.index)
    return df

def load_train():
    print("LOAD TRAIN DATA")
    train = pd.read_csv(Train_Path).dropna()
    train = cleanquestions(train)
    train['question1'].fillna('', inplace=True)
    train['question2'].fillna('', inplace=True)

    return train
def load_test():
    print("LOAD TEST DATA")
    test = pd.read_csv(Test_Path).dropna()
    test = cleanquestions(test)
    test['question1'].fillna('', inplace=True)
    test['question2'].fillna('', inplace=True)

    return test
def splitData(ratio = 0.7): # ratio = train_set, between 0-1.0, default 0.7
    print("SPLIT DATA")
    #time.sleep(1)
    train = load_train()
    X_train, X_test, Y_train, Y_test = train_test_split(train.loc[:,["question1", "question2"]],\
                                                        train["is_duplicate"],\
                                                        train_size=0.7, random_state=42)
    return X_train, X_test, Y_train, Y_test
def trainData():
    print("LOAD TRAIN DATA FOR PREDICT AND SUBMIT")
    train = load_train()
    X = train.loc[:, ["question1", "question2"]]
    Y = train.loc[:]["is_duplicate"]
    return X,Y
def testData():
    test = load_test()
    X_test = test.loc[:,["question1", "question2"]]
    X_ids = test.loc[:]["test_id"]
    return X_test, X_ids

def rprint(str): # Next print overwrites this, i.e use for indicate progress
	sys.stdout.write("PROCESSING: " + str + "            \r")
	sys.stdout.flush()

In [6]:
class BoW:
    name = "BoW"
    cv = 0

    def fit(self, X):
        corpus = pd.concat([X.question1, X.question2])
        self.cv = CountVectorizer(analyzer='word',min_df = 0,
        max_features=5000, ngram_range=(1,1), preprocessor=None, stop_words=None,
        tokenizer=None)
        print("FIT CORPUS ON COUNT VECTORIZER")
        self.cv.fit(corpus)

    def transform(self, X):
        return self.cv.transform(X)
    def getVectorizer(self):
        return self.cv